In [1]:
import numpy as np
import os
import time

In [2]:
st=time.clock()
time.ctime()

'Thu Jul 27 16:31:34 2017'

In [3]:
os.chdir("/Users/sdw/R/")

In [4]:
popsize=100
popsize_half=round(popsize/2)
noff=5
ngen=200
mort_A1=0
mort_A2=0
nloci=2
male_recomb=False
reps=1
recomb_rate=0

A1=1
A2=2
B1=10
B2=20

In [5]:
alle_names=list()
alle_names.append([A1,A2])
alle_names.append([B1,B2])
alle_freq=list()
alle_freq.append([0.5,0.5])
alle_freq.append([0.2,0.8])

In [6]:
alle_freq

[[0.5, 0.5], [0.2, 0.8]]

In [7]:
starting_pop=np.zeros(shape=(popsize,2*nloci+6+1),dtype=np.int) # extra col to coincide with R

In [8]:
starting_pop[0:popsize_half,1]=0 # males
starting_pop[popsize_half:popsize,1]=1 # females
starting_pop[:,2]=1 # population
starting_pop[:,3]=0 # initial generation
starting_pop[:,4]=0 # male parent
starting_pop[:,5]=0 # female parent
starting_pop[:,6]=range(popsize) # id

In [9]:
for k in range(nloci):
    starting_pop[:,6+k+1]=np.random.choice(alle_names[k],p=alle_freq[k],size=popsize,replace=True)
    starting_pop[:,8+k+1]=np.random.choice(alle_names[k],p=alle_freq[k],size=popsize,replace=True)

In [10]:
nextgen=starting_pop

In [11]:
het_tot_A=list()
het_tot_B=list()
gen_tot=list()
alle_freq_tot_A1=list()
alle_freq_tot_A2=list()

In [12]:
g=0
while (True):
    g=g+1
    print("g = {}".format(g))
    male_parents=np.where(starting_pop[:,1]==0)[0]
    female_parents=np.where(starting_pop[:,1]==1)[0]
    parent_matrix=np.zeros(shape=(popsize,2+1),dtype=np.int)
    parent_matrix[:,1]=np.random.choice(male_parents,size=popsize,replace=True)
    parent_matrix[:,2]=np.random.choice(female_parents,size=popsize,replace=True)
    offspring_matrix=np.zeros(shape=(popsize*noff,8+2*nloci+1),dtype=np.int)
    for p in range(popsize):
        offspring_matrix[p*noff:(p+1)*noff,1]=np.random.choice([0,1],size=noff,replace=True)
        offspring_matrix[p*noff:(p+1)*noff,2]=1
        offspring_matrix[p*noff:(p+1)*noff,3]=parent_matrix[p,1]
        offspring_matrix[p*noff:(p+1)*noff,4]=parent_matrix[p,2]
        
        mchrome1=list((starting_pop[parent_matrix[p,1],7],starting_pop[parent_matrix[p,1],8]))
        mchrome2=list((starting_pop[parent_matrix[p,1],9],starting_pop[parent_matrix[p,1],10]))
        fchrome1=list((starting_pop[parent_matrix[p,2],7],starting_pop[parent_matrix[p,2],8]))
        fchrome2=list((starting_pop[parent_matrix[p,2],9],starting_pop[parent_matrix[p,2],10]))
        
        for n in range(noff):
            randad=np.random.rand(1)
            if randad<recomb_rate and male_recomb:
                mctemp1=list((mchrome1[0],mchrome2[1]))
                mctemp2=list((mchrome2[0],mchrome1[1]))
            else:
                mctemp1=list((mchrome1[0],mchrome1[1]))
                mctemp2=list((mchrome2[0],mchrome2[1]))
            malechrome=list((mctemp1,mctemp2))
            offspring_matrix[p*noff+n,8:10]=malechrome[np.random.choice([0,1])]
            
            ranmom=np.random.rand(1)
            if ranmom<recomb_rate:
                fctemp1=list((fchrome1[0],fchrome2[1]))
                fctemp2=list((fchrome2[0],fchrome1[1]))
            else:
                fctemp1=list((fchrome1[0],fchrome1[1]))
                fctemp2=list((fchrome2[0],fchrome2[1]))
            femalechrome=list((fctemp1,fctemp2))
            offspring_matrix[p*noff+n,10:12]=femalechrome[np.random.choice([0,1])]
                
    offspring_matrix[:,6]=range(popsize*noff) # id
    offspring_matrix[:,7]=1 # alive
        
    offspring_matrix[:,12]=np.random.rand(popsize*noff)<mort_A1
    deadA1=(offspring_matrix[:,8]==A1) & (offspring_matrix[:,10]==A1) & offspring_matrix[:,12]
    offspring_matrix[deadA1,7]=0
    offspring_matrix[:,12]=np.random.rand(popsize*noff)<mort_A2
    deadA2=(offspring_matrix[:,8]==A2) & (offspring_matrix[:,10]==A2) & offspring_matrix[:,12]
    offspring_matrix[deadA2,7]=0 
    
    male_off=np.where((offspring_matrix[:,1]==0) & offspring_matrix[:,7])[0]
    female_off=np.where((offspring_matrix[:,1]==1) & offspring_matrix[:,7])[0]
    
    collist=[0,1,2,3,4,5,6,8,9,10,11]
    starting_pop[0:popsize_half]=offspring_matrix[np.random.choice(male_off,size=popsize_half)][:,collist]    
    starting_pop[popsize_half:]=offspring_matrix[np.random.choice(female_off,size=popsize_half)][:,collist]
    
    nextgen=np.vstack((nextgen,starting_pop))
    
    count_A1=sum(starting_pop[:,7]==A1)+sum(starting_pop[:,9]==A1)
    count_A2=sum(starting_pop[:,7]==A2)+sum(starting_pop[:,9]==A2)
    count_B1=sum(starting_pop[:,8]==B1)+sum(starting_pop[:,10]==B1)
    count_B2=sum(starting_pop[:,8]==B2)+sum(starting_pop[:,10]==B2)
    alle_freq_A1=count_A1/(2.0*popsize)
    alle_freq_A2=count_A2/(2.0*popsize)
    alle_freq_B1=count_B1/(2.0*popsize)
    alle_freq_B2=count_B2/(2.0*popsize)
    het_A=1-(np.power(alle_freq_A1,2)+np.power(alle_freq_A2,2))
    het_B=1-(np.power(alle_freq_B1,2)+np.power(alle_freq_B2,2))
    het_tot_A.append(het_A)
    het_tot_B.append(het_B)
    gen_tot.append(g)
    alle_freq_tot_A1.append(alle_freq_A1)
    alle_freq_tot_A2.append(alle_freq_A2)
    if alle_freq_A1==0:
        print("No more A1 after generation {}".format(g))
        break
    if alle_freq_A2==0:
        print("No more A2 after generation {}".format(g))
        break
    

g = 1
g = 2
g = 3
g = 4
g = 5
g = 6
g = 7
g = 8
g = 9
g = 10
g = 11
g = 12
g = 13
g = 14
g = 15
g = 16
g = 17
g = 18
g = 19
g = 20
g = 21
g = 22
g = 23
g = 24
g = 25
g = 26
g = 27
g = 28
g = 29
g = 30
g = 31
g = 32
g = 33
g = 34
g = 35
g = 36
g = 37
g = 38
g = 39
g = 40
g = 41
g = 42
g = 43
g = 44
g = 45
g = 46
g = 47
g = 48
g = 49
g = 50
g = 51
g = 52
g = 53
g = 54
g = 55
g = 56
g = 57
g = 58
g = 59
g = 60
g = 61
g = 62
g = 63
g = 64
g = 65
g = 66
g = 67
g = 68
g = 69
g = 70
g = 71
g = 72
g = 73
g = 74
g = 75
g = 76
g = 77
g = 78
g = 79
g = 80
g = 81
g = 82
g = 83
g = 84
g = 85
g = 86
g = 87
g = 88
g = 89
g = 90
g = 91
g = 92
g = 93
g = 94
g = 95
g = 96
g = 97
g = 98
g = 99
g = 100
g = 101
g = 102
g = 103
g = 104
g = 105
g = 106
g = 107
g = 108
g = 109
g = 110
g = 111
g = 112
g = 113
g = 114
g = 115
g = 116
g = 117
g = 118
g = 119
g = 120
g = 121
g = 122
g = 123
g = 124
g = 125
g = 126
g = 127
g = 128
g = 129
g = 130
g = 131
g = 132
g = 133
g = 134
g = 135
g = 136
g = 137
g = 138
g = 

In [13]:
et=time.clock()
time.ctime()

'Thu Jul 27 16:31:39 2017'

In [14]:
print("Stop the clock {}".format(et-st))

Stop the clock 5.65661


In [15]:
sum(np.array(alle_freq_tot_A1)>0)

156

In [16]:
sum(np.array(alle_freq_tot_A2)>0)

155